In [1]:
import numpy as np
import copy
import itertools
import random
import time
#import math

In [2]:
def tttshow(ox):
    '''显示棋盘，参数是棋盘矩阵'''
    #'''打印列标号'''
    print("{0:^5}".format('x\y'), end='') # 5个字符
    for j in range(ox.shape[1]):
        print("{0:^5}".format(j), end='') # 5个字符          
    print('\n')  # 换行,打印空行        
      
    #'''打印棋盘'''            
    for i in range(ox.shape[0]):        #行数
        #'''打印行标号'''
        print("{0:^5}".format(i), end='') # 5个字符
        #'''打印该行棋盘'''    
        for j in range(ox.shape[1]):
            if ox[i][j] == 0:    #未落子  5个字符
                print("{0:^5}".format('~'), end='')
            elif ox[i][j] == 1:   # x 落子  5个字符
                print("{0:^5}".format('X'), end='')
            elif ox[i][j] == -1:  # o 落子
                print("{0:^5}".format('Q'), end='') 
        print('\n')  
                

def ttt_winner(ox, ninline=3):
    #'''生成棋盘所有ninline维方阵，并判断方阵是否有连成一线的情况'''
    matrixlists = [ox[i:i+ninline, j:j+ninline] 
                    for i in range(ox.shape[0]- ninline +1)
                     for j in range(ox.shape[1]- ninline +1)
                ]
    
    #'''如果出现ninline维方阵返回1或者-1，代表已经有了输赢，返回输赢结果'''
    for nm in matrixlists:
        '''计算方阵连线情况,1代表'x'连成一线，-1代表'o'连成一线，0代表没有'''
        '''把各行列的和组成集合，查找集合中是否存在3或-3，代表连成一行'''
        ox_line = set(nm.sum(axis=1)) | set(nm.sum(axis=0))
        '''第一条对角线的和加入列表'''
        ox_line.add(sum([nm[i, i] for i in range(ninline) ]))
        '''第二条对角线的和加入列表'''
        ox_line.add(sum([nm[ninline-1-i,i] for i in range(ninline) ]))
    
        return -1 if -ninline in ox_line else 1 if ninline in ox_line else 0
   
    
def ttt_simu(ttt, ninline=3, playturn='o', ms=[]):
    '''根据给定棋盘现状和后续下法，返回棋盘最终状态，谁赢或者平局'''
    ox = copy.deepcopy(ttt)
    for i in range(len(ms)):
        ox[ms[i]] = 1 if playturn=='x' else -1     # 更新棋盘   
        if ttt_winner(ox, ninline) == 1:
            return ttt_winner(ox) + (len(ms)-i-1)
        elif ttt_winner(ox, ninline) == -1:
            return ttt_winner(ox) - (len(ms)-i-1)       
        elif ttt_winner(ox, ninline) == 0 :
            playturn = 'o' if playturn=='x' else 'x'
    return 0
    

def ai_montecarlo_search(ox, ninline=3, playturn='o', ox_availables=set(), mounts=1000):
    timebegin = time.time()
    move_value = {x:0 for x in ox_availables}
    for count in range(mounts):
        mv = []
        a_setlist = list(ox_availables)
        ms = len(a_setlist)
        for i in range(ms):
            mv.append(a_setlist.pop(random.randint(0,len(a_setlist)-1)))
        move_value[mv[0]] += ttt_simu(ox, ninline, playturn, mv)
        
    if playturn == 'o':
        minvalue = move_value[min(move_value, key=move_value.get)]
        mvlist = [k for k,v in move_value.items() if v==minvalue ]
    if playturn == 'x':
        maxvalue = move_value[max(move_value, key=move_value.get)]
        mvlist = [k for k,v in move_value.items() if v==maxvalue ]
    print(move_value)
    print(mvlist)
    print(time.time()-timebegin)
    return mvlist[random.randint(0,len(mvlist)-1)] #返回元组(x,y)
    
    
def ai_full_search(ox, ninline=3, playturn='o', ox_availables=set()):
    '''AI走一步，但每走一步都穷尽所有走法，找到最好的,'''
    timebegin = time.time()
    '''生成剩余下法的胜利初始化表'''
    move_value = {x:0 for x in ox_availables}
    #'''一个产生下法序列的生成器函数，穷举直到不再有新的下法序列'''
    #'''使用了排列生成，生成函数是个函数生成器； 返回剩余下法的所有排列'''
    mvs = itertools.permutations(ox_availables) 
    for m in mvs:
        move_value[m[0]] += ttt_simu(ox, ninline, playturn, m)
        
    if playturn == 'o':
        minvalue = move_value[min(move_value, key=move_value.get)]
        mvlist = [k for k,v in move_value.items() if v==minvalue ]
    if playturn == 'x':
        maxvalue = move_value[max(move_value, key=move_value.get)]
        mvlist = [k for k,v in move_value.items() if v==maxvalue ]
    print(move_value)
    print(mvlist)
    print(time.time()-timebegin)
    return mvlist[random.randint(0,len(mvlist)-1)] #返回元组(x,y)

          
def update(ox, player, location):
    '''# player在location(x,y)处落子，更新棋盘，,更新可用位置，计算此时棋局胜负'''
    ox[location] = 1 if player=='x' else -1
    global ttt_availables 
    ttt_availables.remove( location )


In [22]:
''' # 3 x 3 矩阵，值是棋子类型，0空，1代表'x'，-1代表'o'; ''' 
ttt = np.zeros((9, 9), dtype=int )
ninline = 5
ttt_availables = set( [(i,j) for i in range(9) for j in range(9)] )  
player = 'x'

In [23]:
while True :

    tttshow(ttt)

    if player == 'x':
        #print('请选手选择落子处的数字，如23代表在第2行第3列落子：', end='')
        #xy = input()
        #playlocation = (int(xy[0]), int(xy[1]) )
        playlocation = ai_montecarlo_search(ttt, ninline, player, ttt_availables)
        update(ttt, player, playlocation)
        if ttt_winner(ttt, ninline) == 1:
            print('选手X 赢')
            tttshow(ttt)
            break
        if ttt_availables == set():
            print('平局')
            tttshow(ttt)
            break
        player = 'o'

    if player == 'o':
        #playlocation = ai_full_search(ttt, player, ttt_availables)
        playlocation = ai_montecarlo_search(ttt, ninline, player, ttt_availables)
        update(ttt, player, playlocation)
        if ttt_winner(ttt, ninline) == -1:
            print('计算机Q 赢')
            tttshow(ttt)
            break 
        if ttt_availables == set():
            print('平局')
            tttshow(ttt)
            break
        player = 'x'


 x\y   0    1    2    3    4    5    6    7    8  

  0    ~    ~    ~    ~    ~    ~    ~    ~    ~  

  1    ~    ~    ~    ~    ~    ~    ~    ~    ~  

  2    ~    ~    ~    ~    ~    ~    ~    ~    ~  

  3    ~    ~    ~    ~    ~    ~    ~    ~    ~  

  4    ~    ~    ~    ~    ~    ~    ~    ~    ~  

  5    ~    ~    ~    ~    ~    ~    ~    ~    ~  

  6    ~    ~    ~    ~    ~    ~    ~    ~    ~  

  7    ~    ~    ~    ~    ~    ~    ~    ~    ~  

  8    ~    ~    ~    ~    ~    ~    ~    ~    ~  

{(7, 3): -24, (0, 7): -24, (1, 6): -23, (3, 7): 16, (2, 5): -28, (8, 5): 72, (5, 8): 54, (4, 0): -58, (6, 7): 36, (5, 5): -26, (7, 6): 9, (0, 4): 41, (1, 1): 124, (3, 2): -25, (2, 6): -42, (8, 2): -34, (4, 5): -25, (6, 0): 41, (7, 5): 15, (0, 1): 91, (3, 1): 37, (7, 8): 14, (2, 1): -43, (5, 1): -18, (7, 2): -4, (1, 5): -14, (3, 6): -28, (2, 2): 71, (8, 6): -50, (4, 1): -86, (6, 4): 14, (5, 4): 66, (7, 1): 13, (0, 5): 12, (1, 0): -18, (0, 8): -52, (3, 5): 86, (2, 7): 16, (8, 3

{(7, 3): 34, (0, 7): 88, (1, 6): -84, (3, 7): 85, (2, 5): 202, (8, 5): -15, (5, 8): 11, (4, 0): 90, (6, 7): 39, (5, 5): 27, (7, 6): 35, (0, 4): -10, (3, 2): -47, (8, 2): 26, (4, 5): 37, (6, 0): 115, (7, 5): 33, (0, 1): 104, (3, 1): -29, (7, 8): 8, (2, 1): 18, (7, 2): 141, (1, 5): 27, (3, 6): 11, (2, 2): 118, (8, 6): -8, (4, 1): 49, (6, 4): 99, (5, 4): 76, (7, 1): 36, (0, 5): 126, (1, 0): 55, (0, 8): 61, (3, 5): 19, (2, 7): -54, (8, 3): 74, (4, 6): -14, (6, 1): 103, (5, 7): 61, (7, 4): 84, (0, 2): 47, (1, 3): 36, (4, 8): 154, (3, 0): 111, (2, 8): 49, (8, 0): 21, (6, 2): 40, (5, 0): 72, (1, 4): -61, (2, 3): 5, (4, 2): -32, (6, 5): 32, (5, 3): 129, (7, 0): 130, (0, 6): -5, (1, 7): -58, (3, 4): 39, (2, 4): 18, (8, 4): 39, (4, 7): 64, (6, 6): 140, (5, 6): -47, (7, 7): -11, (0, 3): -29, (3, 3): 39, (8, 1): 118, (4, 4): 25, (6, 3): 61, (0, 0): 124, (3, 8): 5, (2, 0): 121, (1, 8): 72, (8, 8): 60, (4, 3): 67}
[(1, 6)]
9.022061109542847
 x\y   0    1    2    3    4    5    6    7    8  

  0    

{(7, 3): 7, (0, 7): 7, (3, 7): 41, (2, 5): 66, (8, 5): 100, (4, 0): 132, (6, 7): 86, (5, 5): 63, (7, 6): -12, (3, 2): 16, (8, 2): -42, (4, 5): 78, (6, 0): 56, (7, 5): -15, (7, 8): -22, (2, 1): 47, (7, 2): 21, (1, 5): -30, (3, 6): 20, (2, 2): 61, (8, 6): 77, (4, 1): 45, (6, 4): 11, (5, 4): 19, (7, 1): 62, (1, 0): 2, (0, 8): 15, (3, 5): -4, (2, 7): 2, (8, 3): 52, (4, 6): 37, (6, 1): 10, (5, 7): 47, (7, 4): -21, (1, 3): 89, (4, 8): -9, (3, 0): 38, (2, 8): 56, (8, 0): -6, (6, 2): 124, (5, 0): 16, (1, 4): 20, (2, 3): 120, (4, 2): 45, (6, 5): 61, (5, 3): 32, (7, 0): 147, (0, 6): 44, (1, 7): 18, (3, 4): -11, (2, 4): 102, (8, 4): -13, (4, 7): 55, (6, 6): 84, (5, 6): -14, (7, 7): 53, (0, 3): 41, (3, 3): 144, (8, 1): 29, (4, 4): 3, (6, 3): 44, (0, 0): 125, (3, 8): 32, (2, 0): 25, (1, 8): -25, (8, 8): 20, (4, 3): 85}
[(7, 0)]
8.476187705993652
{(7, 3): 13, (0, 7): -14, (3, 7): 90, (2, 5): 72, (8, 5): 126, (4, 0): 71, (6, 7): 85, (5, 5): -9, (7, 6): -33, (3, 2): -3, (8, 2): 19, (4, 5): 2, (6, 0): 

{(7, 3): 50, (0, 7): 58, (3, 7): -8, (2, 5): 30, (8, 5): 28, (4, 0): 49, (6, 7): 9, (5, 5): 137, (7, 6): -8, (3, 2): -14, (8, 2): 45, (4, 5): 3, (6, 0): -8, (7, 5): 7, (7, 8): 10, (2, 1): 57, (7, 2): 12, (1, 5): 42, (3, 6): 66, (8, 6): -1, (4, 1): 55, (6, 4): 9, (5, 4): -36, (7, 1): 32, (1, 0): 118, (0, 8): 0, (3, 5): 41, (2, 7): 73, (8, 3): 29, (4, 6): -1, (6, 1): 63, (5, 7): 3, (7, 4): 63, (1, 3): 8, (4, 8): 56, (2, 8): 16, (8, 0): 39, (6, 2): 87, (5, 0): -5, (2, 3): 28, (4, 2): 46, (6, 5): 30, (5, 3): 19, (0, 6): 9, (1, 7): -17, (3, 4): 22, (8, 4): 14, (4, 7): 13, (6, 6): 69, (5, 6): 46, (7, 7): 127, (0, 3): 129, (8, 1): -17, (6, 3): 50, (0, 0): 8, (3, 8): 6, (1, 8): 29, (8, 8): -8, (4, 3): 125}
[(5, 5)]
8.455790758132935
{(7, 3): 12, (0, 7): -24, (3, 7): 46, (2, 5): 11, (8, 5): 50, (4, 0): 0, (6, 7): 31, (7, 6): -33, (3, 2): 21, (8, 2): -19, (4, 5): 52, (6, 0): 31, (7, 5): 19, (7, 8): 140, (2, 1): -20, (7, 2): 65, (1, 5): 52, (3, 6): 44, (8, 6): -5, (4, 1): 0, (6, 4): 82, (5, 4): -

{(7, 3): -10, (0, 7): -78, (3, 7): -16, (2, 5): -22, (8, 5): -14, (6, 7): -56, (3, 2): -16, (8, 2): 0, (4, 5): -38, (6, 0): -42, (7, 5): -20, (7, 8): -22, (2, 1): -34, (7, 2): -16, (1, 5): -28, (3, 6): -32, (8, 6): -10, (4, 1): 0, (6, 4): 0, (5, 4): -34, (7, 1): -98, (0, 8): -4, (3, 5): -22, (2, 7): -54, (8, 3): -24, (4, 6): -74, (6, 1): -24, (5, 7): -42, (7, 4): -30, (1, 3): -28, (4, 8): -36, (8, 0): -20, (6, 2): 0, (5, 0): 0, (2, 3): -112, (6, 5): -36, (5, 3): -20, (0, 6): -14, (1, 7): -2, (3, 4): -58, (8, 4): 0, (4, 7): -50, (6, 6): -24, (5, 6): -2, (7, 7): -74, (0, 3): -90, (6, 3): -4, (3, 8): -6, (1, 8): 0, (8, 8): -12}
[(2, 3)]
6.595204591751099
 x\y   0    1    2    3    4    5    6    7    8  

  0    Q    X    Q    ~    X    Q    ~    ~    ~  

  1    X    X    X    ~    Q    ~    Q    ~    ~  

  2    X    ~    Q    Q    Q    ~    Q    ~    X  

  3    X    Q    ~    Q    ~    ~    ~    ~    ~  

  4    X    ~    X    Q    X    ~    ~    ~    ~  

  5    ~    X    X    ~    ~

{(7, 3): 0, (0, 7): 0, (3, 7): 0, (2, 5): 0, (6, 7): 0, (3, 2): 0, (8, 2): 0, (4, 5): 0, (6, 0): 0, (7, 5): 0, (7, 8): 0, (2, 1): 0, (7, 2): 0, (1, 5): 0, (8, 6): 0, (4, 1): 0, (5, 4): 0, (7, 1): 0, (3, 5): 0, (8, 3): 0, (4, 6): 0, (6, 1): 0, (5, 7): 0, (7, 4): 0, (4, 8): 0, (8, 0): 0, (6, 2): 0, (5, 0): 0, (6, 5): 0, (5, 3): 0, (0, 6): 0, (1, 7): 0, (3, 4): 0, (8, 4): 0, (4, 7): 0, (6, 6): 0, (5, 6): 0, (7, 7): 0, (6, 3): 0, (3, 8): 0, (1, 8): 0, (8, 8): 0}
[(7, 3), (0, 7), (3, 7), (2, 5), (6, 7), (3, 2), (8, 2), (4, 5), (6, 0), (7, 5), (7, 8), (2, 1), (7, 2), (1, 5), (8, 6), (4, 1), (5, 4), (7, 1), (3, 5), (8, 3), (4, 6), (6, 1), (5, 7), (7, 4), (4, 8), (8, 0), (6, 2), (5, 0), (6, 5), (5, 3), (0, 6), (1, 7), (3, 4), (8, 4), (4, 7), (6, 6), (5, 6), (7, 7), (6, 3), (3, 8), (1, 8), (8, 8)]
5.452093601226807
 x\y   0    1    2    3    4    5    6    7    8  

  0    Q    X    Q    Q    X    Q    ~    ~    X  

  1    X    X    X    X    Q    ~    Q    ~    ~  

  2    X    ~    Q    Q   

{(7, 3): 0, (0, 7): 0, (2, 5): 0, (6, 7): 0, (4, 5): 0, (7, 5): 0, (7, 8): 0, (2, 1): 0, (7, 2): 0, (1, 5): 0, (8, 6): 0, (4, 1): 0, (7, 1): 0, (3, 5): 0, (8, 3): 0, (5, 7): 0, (7, 4): 0, (4, 8): 0, (8, 0): 0, (6, 2): 0, (5, 0): 0, (6, 5): 0, (5, 3): 0, (1, 7): 0, (3, 4): 0, (8, 4): 0, (4, 7): 0, (6, 6): 0, (5, 6): 0, (7, 7): 0, (6, 3): 0, (3, 8): 0, (8, 8): 0}
[(7, 3), (0, 7), (2, 5), (6, 7), (4, 5), (7, 5), (7, 8), (2, 1), (7, 2), (1, 5), (8, 6), (4, 1), (7, 1), (3, 5), (8, 3), (5, 7), (7, 4), (4, 8), (8, 0), (6, 2), (5, 0), (6, 5), (5, 3), (1, 7), (3, 4), (8, 4), (4, 7), (6, 6), (5, 6), (7, 7), (6, 3), (3, 8), (8, 8)]
4.284219741821289
{(7, 3): 0, (0, 7): 0, (2, 5): 0, (6, 7): 0, (4, 5): 0, (7, 5): 0, (7, 8): 0, (7, 2): 0, (1, 5): 0, (8, 6): 0, (4, 1): 0, (7, 1): 0, (3, 5): 0, (8, 3): 0, (5, 7): 0, (7, 4): 0, (4, 8): 0, (8, 0): 0, (6, 2): 0, (5, 0): 0, (6, 5): 0, (5, 3): 0, (1, 7): 0, (3, 4): 0, (8, 4): 0, (4, 7): 0, (6, 6): 0, (5, 6): 0, (7, 7): 0, (6, 3): 0, (3, 8): 0, (8, 8): 0}


{(0, 7): 0, (2, 5): 0, (6, 7): 0, (4, 5): 0, (7, 8): 0, (7, 2): 0, (8, 6): 0, (7, 1): 0, (3, 5): 0, (8, 3): 0, (4, 8): 0, (8, 0): 0, (5, 0): 0, (6, 5): 0, (5, 3): 0, (8, 4): 0, (4, 7): 0, (6, 6): 0, (5, 6): 0, (7, 7): 0, (6, 3): 0, (3, 8): 0, (8, 8): 0}
[(0, 7), (2, 5), (6, 7), (4, 5), (7, 8), (7, 2), (8, 6), (7, 1), (3, 5), (8, 3), (4, 8), (8, 0), (5, 0), (6, 5), (5, 3), (8, 4), (4, 7), (6, 6), (5, 6), (7, 7), (6, 3), (3, 8), (8, 8)]
2.8637166023254395
{(0, 7): 0, (2, 5): 0, (6, 7): 0, (4, 5): 0, (7, 8): 0, (7, 2): 0, (8, 6): 0, (7, 1): 0, (3, 5): 0, (8, 3): 0, (4, 8): 0, (8, 0): 0, (5, 0): 0, (6, 5): 0, (5, 3): 0, (8, 4): 0, (6, 6): 0, (5, 6): 0, (7, 7): 0, (6, 3): 0, (3, 8): 0, (8, 8): 0}
[(0, 7), (2, 5), (6, 7), (4, 5), (7, 8), (7, 2), (8, 6), (7, 1), (3, 5), (8, 3), (4, 8), (8, 0), (5, 0), (6, 5), (5, 3), (8, 4), (6, 6), (5, 6), (7, 7), (6, 3), (3, 8), (8, 8)]
2.7232534885406494
 x\y   0    1    2    3    4    5    6    7    8  

  0    Q    X    Q    Q    X    Q    Q    ~    X  


{(0, 7): 0, (6, 7): 0, (7, 2): 0, (3, 5): 0, (4, 8): 0, (6, 5): 0, (6, 3): 0, (3, 8): 0, (8, 8): 0}
[(0, 7), (6, 7), (7, 2), (3, 5), (4, 8), (6, 5), (6, 3), (3, 8), (8, 8)]
1.1727933883666992
{(0, 7): 0, (6, 7): 0, (7, 2): 0, (3, 5): 0, (6, 5): 0, (6, 3): 0, (3, 8): 0, (8, 8): 0}
[(0, 7), (6, 7), (7, 2), (3, 5), (6, 5), (6, 3), (3, 8), (8, 8)]
1.1518607139587402
 x\y   0    1    2    3    4    5    6    7    8  

  0    Q    X    Q    Q    X    Q    Q    Q    X  

  1    X    X    X    X    Q    Q    Q    X    Q  

  2    X    X    Q    Q    Q    Q    Q    Q    X  

  3    X    Q    X    Q    X    ~    Q    X    ~  

  4    X    X    X    Q    X    X    Q    X    X  

  5    Q    X    X    Q    X    X    X    Q    X  

  6    Q    X    Q    ~    X    ~    Q    ~    Q  

  7    X    Q    ~    Q    Q    X    Q    Q    X  

  8    X    Q    Q    X    X    X    Q    Q    ~  

{(6, 7): 0, (7, 2): 0, (3, 5): 0, (6, 5): 0, (6, 3): 0, (3, 8): 0, (8, 8): 0}
[(6, 7), (7, 2), (3, 5), (6, 5), (6, 